In [61]:
import sys
import cmath
import numpy as np
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt
from qci_client import QciClient, JobStatus
import os
import pkg_resources

import dimod
import math
import pandas as pd
from itertools import permutations 
import operator
from operator import itemgetter
from dimod.reference.samplers import ExactSolver
from minorminer import find_embedding

In [80]:
def CRU(n):
    theta = math.pi * 2 / n
 
    roots = []
    for k in range(0, n):
        # calculate the real and imaginary part of root
        real = math.cos(k * theta)
        img = math.sin(k * theta)
        z = complex(real,img)
        roots.append(z)
    return roots
        
def QFT(n):
    QFT_Matrix = [[]]*n
    for i in range(n):
        QFT_Matrix[i] = np.power(CRU(n), i)
    return QFT_Matrix
    
    
    
A = QFT(4)
print(A)
graph_A = np.abs(A)
nodes = []
edges = []
for i in range(len(A)):
    for j in range(len(A[i])):
        if i == j:        
            dict0 = {'i':i,'j':j, 'val':A[i][j]}
            nodes.append(dict0)
        else:
            dict1 = {'i':i,'j':j, 'val':A[i][j]}
            edges.append(dict1)

[array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]), array([ 1.0000000e+00+0.0000000e+00j,  6.1232340e-17+1.0000000e+00j,
       -1.0000000e+00+1.2246468e-16j, -1.8369702e-16-1.0000000e+00j]), array([ 1.+0.0000000e+00j, -1.+1.2246468e-16j,  1.-2.4492936e-16j,
       -1.+3.6739404e-16j]), array([ 1.0000000e+00+0.0000000e+00j, -1.8369702e-16-1.0000000e+00j,
       -1.0000000e+00+3.6739404e-16j,  5.5109106e-16+1.0000000e+00j])]


In [81]:
nodges = nodes + edges
print(nodges)       
Graph = nx.from_numpy_array(A)
nx.draw(Graph)

qc = QciClient()


# Reads the file and returns an array of lines found in the file        
# example of representing a qubo as a dict
qubo_dict = {"file_type": "qubo", "file_name": "qubo.json",
             "data": nodges,
             "num_variables": len(A)}


[{'i': 0, 'j': 0, 'val': (1+0j)}, {'i': 1, 'j': 1, 'val': (6.123233995736766e-17+1j)}, {'i': 2, 'j': 2, 'val': (1-2.4492935982947064e-16j)}, {'i': 3, 'j': 3, 'val': (5.51091059616309e-16+1j)}, {'i': 0, 'j': 1, 'val': (1+0j)}, {'i': 0, 'j': 2, 'val': (1+0j)}, {'i': 0, 'j': 3, 'val': (1+0j)}, {'i': 1, 'j': 0, 'val': (1+0j)}, {'i': 1, 'j': 2, 'val': (-1+1.2246467991473532e-16j)}, {'i': 1, 'j': 3, 'val': (-1.8369701987210297e-16-1j)}, {'i': 2, 'j': 0, 'val': (1+0j)}, {'i': 2, 'j': 1, 'val': (-1+1.2246467991473532e-16j)}, {'i': 2, 'j': 3, 'val': (-1+3.6739403974420594e-16j)}, {'i': 3, 'j': 0, 'val': (1+0j)}, {'i': 3, 'j': 1, 'val': (-1.8369701987210297e-16-1j)}, {'i': 3, 'j': 2, 'val': (-1+3.6739403974420594e-16j)}]


AttributeError: 'list' object has no attribute 'ndim'

In [76]:
job_type = "sample-qubo"
job_params = {}     # dict of additional params to be passed to job submission in "params" key

# nsamples = None
nsamples = 5
if nsamples:
    job_params["nsamples"] = nsamples

# see quoir_svc/consumers/base.py:self.QPU_DEVICE_NAMES for names that can be used for sampler
# current known valid sampler names:

# sampler = None
# sampler = "csample"
# sampler = "braket_simulator_sv1"
# sampler = "braket_dwave_advantage6"
sampler = "eqc1"
# sampler = "eqc2"
# sampler = "eqc3"

if sampler == "eqc2" or sampler == "eqc3":
    print(f"Error: sampler=\"{sampler}\" cannot run sample-qubo on qdit systems")
    exit(1)

# pick one of the two qubo representations and upload it as a file
qubo_file_id = qc.upload_file(data=qubo_dict)["file_id"]
# qubo_file_id = qc.upload_file(data=qubo_sp)["file_id"]        # not currently working in qci-client

if sampler:
    job_params["sampler_type"] = sampler
if 'solution_type' in vars():
    if solution_type:
        job_params["solution_type"] = solution_type

print(f'sampler="{sampler}" using environment: {os.environ["QCI_API_URL"]}')
if 'solution_type' in vars():
    if solution_type:
        print(f'\t solution_type: {solution_type}')

qubo_job_body = qc.build_job_body(job_type=job_type, job_params=job_params, qubo_file_id=qubo_file_id,
                                  job_name="qubo_job_0", job_tags=[])

result = qc.process_job(job_type=job_type, job_body=qubo_job_body, wait=True)
print(result)

if result['job_info']['details']['status'] == "ERROR":
    print(result['job_info']['details']['status'])
    print(result['job_info']['results']['error'])




Upload of file part 0 generated exception 'Object of type complex128 is not JSON serializable'.


TypeError: Object of type complex128 is not JSON serializable